In [ ]:
from tensorflow import keras
import numpy as np
import tensorflow as tf

reconstructed_model = keras.models.load_model("model_Final_v1.3.h5")



reconstructed_model.summary()
print(tf.__version__)


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 2, 4, 32)          64        
                                                                 
 conv2d_13 (Conv2D)          (None, 2, 3, 32)          2080      
                                                                 
 conv2d_14 (Conv2D)          (None, 1, 2, 32)          4128      
                                                                 
 flatten_4 (Flatten)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 8)                

In [ ]:

input_str = '1378;105;972;38;87;2;73;93'
arr = list(map(float, input_str.split(';')))
print(np.array(arr))
mean = np.array([1991.033102, 979.100292, 1384.605909, 690.083826, 202.240331, 50218.089443, 303.198568, 152.770388])
std = np.array([1158.654284, 1460.097603, 806.427103, 1100.119750, 167.640898,268559.217730,386.393986, 135.929749])
arr = (arr- mean) / std
arr_0 = np.stack((arr[0],arr[2],arr[4],arr[6]), axis=0)
arr_1 = np.stack((arr[1],arr[3], arr[5],arr[7]), axis=0)
x_arr = np.stack((arr_0,arr_1), axis=0)
print(x_arr)

print(x_arr.reshape(1,2,4,1))
y_prob = reconstructed_model.predict(x_arr.reshape(1,2,4,1))
y_classes = y_prob.argmax()
print(y_prob)
print(y_classes)

[1378.  105.  972.   38.   87.    2.   73.   93.]
[[-0.52909061 -0.51164688 -0.68742373 -0.59576126]
 [-0.59865881 -0.59273895 -0.1869833  -0.43971528]]
[[[[-0.52909061]
   [-0.51164688]
   [-0.68742373]
   [-0.59576126]]

  [[-0.59865881]
   [-0.59273895]
   [-0.1869833 ]
   [-0.43971528]]]]
1/1 [==============================] - 0s 131ms/step
[[7.8270929e-08 3.0368696e-08 5.8848183e-08 5.2309019e-08 2.5802500e-08
  5.4409270e-08 3.0211544e-01 6.9788420e-01]]
7


In [ ]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(reconstructed_model) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model_final_v1.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model)

=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the CONV_2D op takes
tensor #0 and tensor #4 and tensor #1 as input and produces tensor #12 as output.

Subgraph#0 main(T#0) -> [T#18]
  Op#0 CONV_2D(T#0, T#4, T#1) -> [T#12]
  Op#1 CONV_2D(T#12, T#5, T#2) -> [T#13]
  Op#2 CONV_2D(T#13, T#6, T#3) -> [T#14]
  Op#3 RESHAPE(T#14, T#9[-1, 64]) -> [T#15]
  Op#4 FULLY_CONNECTED(T#15, T#10, T#8) -> [T#16]
  Op#5 FULLY_CONNECTED(T#16, T#11, T#7) -> [T#17]
  Op#6 SOFTMAX(T#17) -> [T#18]

Tensors of Subgraph#0
  T#0(serving_default_conv2d_12_input:0) shape_signature:[-1, 2, 4, 1], type:FLOAT32
  T#1(sequential_4/conv2d_12/BiasAdd/ReadVariableOp) shape:[32], type:FLOAT32 RO 128 bytes, buffer: 2, data:[0.169275, 0.123473, -0.000977548, -0.00709174, 0.145095, ...]
  T#2(sequential_4/conv2d_13/BiasAdd/ReadVariableOp) shape:[32], type:FLOAT32 RO 128 bytes, buffer: 3, data:[0.162775, -0.0

In [ ]:
# Load TFLite model and allocate tensors.

interpreter = tf.lite.Interpreter(model_path="model_final_v1.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)

print(output_details)

# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(x_arr.reshape(1,2,4,1),dtype=np.float32)
print(input_data)
print(input_shape)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data[0])

[{'name': 'serving_default_conv2d_12_input:0', 'index': 0, 'shape': array([1, 2, 4, 1], dtype=int32), 'shape_signature': array([-1,  2,  4,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 18, 'shape': array([1, 8], dtype=int32), 'shape_signature': array([-1,  8], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[[[[-0.5290906 ]
   [-0.51164687]
   [-0.6874237 ]
   [-0.59576124]]

  [[-0.5986588 ]
   [-0.5927389 ]
   [-0.1869833 ]
   [-0.4397153 ]]]]
[1 2 4 1]
[7.8270908e-08 3.0368689e-08 5.8848165e-08 5.2309204e-08 2.5802491e-08
 5.4409366e-08 3.0211562e-01 6.9788402e-01]


In [ ]:
y_testclasses = output_data.argmax()
print(y_testclasses)
print(type(y_testclasses))


7
<class 'numpy.int64'>
